# *Retele de calculatoare - Tema 1*

 # **Biti de paritate bidimensionala**

In [67]:
import math
import numpy as np
from numpy import ndarray
import random

1.	Se introduce de la tastatura un sir format dintr-un numar multiplu de 7 caractere binare (Se vor face verificari in acest sens). 

In [68]:
def remove_first_zeros(input_str:str) -> str:
    if input_str.startswith('0'):
        return input_str.lstrip('0')
    else:
        return input_str

def binary_characters(input_str: str) -> bool:
    return all(char == '0' or char == '1' for char in input_str)

def validate_input(input_str: str) -> bool:
    return len(input_str) % 7 == 0 and binary_characters(input_str)

message_str: str = '10101110101110'
message_str = remove_first_zeros(message_str)
ok: bool = False

while not ok:
    if not validate_input(message_str):
        new_message = input("Invalid input\nEnter new message: ")
        message_str = new_message
    else:
        print("Valid input.")
        print(message_str)
        ok = True

Valid input.
10101110101110


2.	Mesajul va fi pozitionat intr-o matrice cu 7 coloane si numar de linii variabil (in functie de lungimea mesajului introdus).

In [69]:
def convert_input_to_matrix(message: str, columns:int) -> ndarray:
    if validate_input(message) is False:
        raise ValueError("Invalid message length.")
    else:
        mat = np.array(list(message),  dtype=int).reshape(len(message)//columns, columns)
        return mat

message:ndarray = convert_input_to_matrix(message_str, 7)
print(message)

[[1 0 1 0 1 1 1]
 [0 1 0 1 1 1 0]]


3.	Se vor calcula bitii de paritate pe fiecare linie si fiecare coloana astfel:
-	pentru  numar par de 1 pe o linie/coloana  se va adauga un 0 pe ultima coloana/linie;
-	pentru un numar impar de 1 se va adauga un 1;
-	bitul semnificativ este bitul din coltul dreapta-jos al matricei.

4.	Se va afisa matricea rezultata.

In [70]:
def message_matrix(message:ndarray) -> ndarray:
    new_mat = np.zeros((message.shape[0] + 1, message.shape[1] + 1), dtype=int)
    new_mat[:-1, :-1] = message
    new_mat[-1, :-1] = new_mat[:-1, :-1].sum(axis=0) % 2
    new_mat[:, -1] = np.sum(new_mat[:, :-1], axis=1) % 2
    return new_mat

n_message:ndarray = message_matrix(message)
print(n_message)

[[1 0 1 0 1 1 1 1]
 [0 1 0 1 1 1 0 0]
 [1 1 1 1 0 0 1 1]]


5.	Se va simula coruperea mesajului prin generarea aleatoare a unei pozitii a unui caracter binar al mesajului. In functie de pozitia generata, se va modifica bitul de pe acea pozitie. 

In [71]:
def corrupt_message(message:ndarray) -> ndarray:
    rows, cols = message.shape
    row, col = random.randint(0, rows - 2), random.randint(0, cols - 2)
    print("Corrupted Position:", (row, col))
    
    if(message[row,col] == 1):
        message[row,col] = 0
    else:
        message[row,col] = 1
    
    return message

corrupted_message = corrupt_message(n_message)
print(corrupted_message)


Corrupted Position: (0, 5)
[[1 0 1 0 1 0 1 1]
 [0 1 0 1 1 1 0 0]
 [1 1 1 1 0 0 1 1]]


6.	Destinatia va reface calculul bitilor de paritate bidimensionala si va identifica bitul corupt.
7.	Destinatia va afisa apoi pozitia corupta.

In [72]:
def two_dimensional_parity_check(message: ndarray) -> tuple:
    rows, cols = message.shape
    row_index, col_index = None, None
    
    for row in range(rows - 1): # check for rows
        row_ones = sum(message[row, :cols - 1])  
        if row_ones % 2 != message[row, cols - 1]:  
            row_index = row
            break
    
    for col in range(cols - 1): # check for columns
        col_ones = sum(message[:rows - 1, col]) 
        if col_ones % 2 != message[rows - 1, col]: 
            col_index = col
            break
    
    return (row_index, col_index) if row_index is not None and col_index is not None else None

def message_display(message:ndarray) -> None:
    if(two_dimensional_parity_check(message)):
        print("Corrupted message detected at position:", two_dimensional_parity_check(message), "\n")
    else:
        print("No corrupted message detected.\n")

print(message_matrix(message))
message_display(message_matrix(message))
print(corrupted_message)
message_display(corrupted_message)


[[1 0 1 0 1 1 1 1]
 [0 1 0 1 1 1 0 0]
 [1 1 1 1 0 0 1 1]]
No corrupted message detected.

[[1 0 1 0 1 0 1 1]
 [0 1 0 1 1 1 0 0]
 [1 1 1 1 0 0 1 1]]
Corrupted message detected at position: (0, 5) 



# **CRC (Cyclic Redundancy Check)**

1.	Se introduce de la tastatura un sir de caractere binare si un polinom generator (cu coeficienti 0 si 1).

2.	Se fac urmatoarele verificari: sirurile sa fie binare si lungimea mesajului sa fie mai mare decat numarul de coeficienti ai polinomului generator.

3.	Se extinde mesajul cu un numar de 0-uri egal cu gradul polinomului introdus.

4.	Se efectueaza succesiv operatii de XOR intre mesajul extins si coeficientii polinomului pana cand lungimea restului obtinut este strict mai mica decat lungimea sirului de coeficienti.
5.	Se vor afisa rezultatele intermediare ale operatiei de XOR.

6.	Se executa iarasi operatia de sau exclusiv intre mesajul extins si restul final obtinut, dar pozitionarea restului se va face sub finalul mesajului extins. 
7.	Acest rezultat se va afisa.